In [ ]:
import os
import sys
import time
import json
import random
import itertools
from itertools import cycle, islice

# Numerical & Data Handling
import numpy as np
import pandas as pd
from pandas import DataFrame as df

# Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Machine Learning & Preprocessing
from sklearn import datasets, metrics, linear_model, svm
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, KFold,
    RandomizedSearchCV, GridSearchCV
)

# Classifier
from sklearn.ensemble import ExtraTreesClassifier as et

# Metrics
from sklearn.metrics import (
    roc_curve, auc, roc_auc_score,
    precision_recall_curve, average_precision_score,
    brier_score_loss,
    f1_score, accuracy_score, confusion_matrix
)

import shap

In [2]:

ABP_Outcome=pd.read_csv('/opt/genomics/commonfilesharenonphi/208pomc/Feature_ABP_Outcomes.csv')
ABP_Feature=pd.read_csv('/opt/genomics/commonfilesharenonphi/208pomc/ABP_FEATURE_LIBRARY_MLORD_DATASET.csv')
#Feature Column1='StudyID'
PPG_Feature=pd.read_csv('/opt/genomics/commonfilesharenonphi/208pomc/PPG_FEATURE_LIBRARY_MLORD_DATASET.csv')
PPG_Outcome=pd.read_csv('/opt/genomics/commonfilesharenonphi/208pomc/Feature_PPG_Outcomes.csv')
#Outcome Column2='Study_ID' 

In [ ]:
#ABP Prep
ABP=ABP_Feature.merge(ABP_Outcome[['StudyID','POMC']],left_on='Study_ID',right_on='StudyID')
ABP=ABP.replace("Male",1).replace("Female",0).replace("Unknown",1)
#print(ABP['sex'].value_counts())
Features=ABP.columns.to_list()[1:len(ABP.columns)-2]
X_train, X_temp, y_train, y_temp = train_test_split(ABP[Features],ABP['POMC'],test_size=0.3, stratify=ABP['POMC'])
X_validation, X_test, y_validation, y_test = train_test_split(X_temp,y_temp,test_size=0.66, stratify=y_temp)

In [ ]:
#PPG Prep
PPG=PPG_Feature.merge(PPG_Outcome[['StudyID','POMC']],left_on='Study_ID',right_on='StudyID')
PPG=PPG.replace("Male",1).replace("Female",0).replace("Unknown",1)
#print(PPG['sex'].value_counts())
Features=PPG.columns.to_list()[1:len(PPG.columns)-2]
X_train, X_temp, y_train, y_temp = train_test_split(PPG[Features],PPG['POMC'],test_size=0.3, stratify=PPG['POMC'])
X_validation, X_test, y_validation, y_test = train_test_split(X_temp,y_temp,test_size=0.66, stratify=y_temp)

In [27]:
#Combined
ComboTemp=ABP_Feature.merge(ABP_Outcome[['StudyID','POMC']],left_on='Study_ID',right_on='StudyID',how='inner')
ComboTemp=ComboTemp.drop(columns=['StudyID','age','sex'])
Combo=ComboTemp.merge(PPG_Feature, on='Study_ID', how='inner')
Combo=Combo.replace("Male",1).replace("Female",0).replace("Unknown",1)
#print(Combo['sex'].value_counts())
X=Combo.drop(columns=['POMC'])
Features=X.columns.to_list()[1:len(X.columns)]
X_train, X_temp, y_train, y_temp = train_test_split(X[Features],Combo['POMC'],test_size=0.3, stratify=Combo['POMC'])
X_validation, X_test, y_validation, y_test = train_test_split(X_temp,y_temp,test_size=0.66, stratify=y_temp)

In [ ]:
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seeds(42)
model = et()
parameter_space = {
    'n_estimators':[100,200],
    'criterion':['entropy','gini','log_loss'],
    'min_samples_split':[1,2,3],
    'min_samples_leaf':[1,2],
    'max_depth':[None,2,3],
    'max_features':[None,'sqrt','log2']
}

folds = 4

skf = StratifiedKFold(n_splits=folds, shuffle = True)
random_search = GridSearchCV(model, parameter_space, scoring=['roc_auc','average_precision'],verbose=2, return_train_score=True, cv=skf.split(X_train,y_train),refit='average_precision')
random_search.fit(X_train, y_train)
print("Best Parameters: ", random_search.best_params_)
df=pd.DataFrame(random_search.cv_results_)
df[df["rank_test_roc_auc"]==1][["params","mean_test_roc_auc","std_test_roc_auc","mean_test_average_precision","std_test_average_precision"]]

In [ ]:
best_model = random_search.best_estimator_


y_pred = best_model.predict(X_validation)
y_prob = best_model.predict_proba(X_validation)[:,1]
cm=confusion_matrix(y_validation, y_pred)
tn, fp, fn, tp = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
roc_auc=roc_auc_score(y_validation, y_prob)
f1 = f1_score(y_validation,y_pred)
accuracy = accuracy_score(y_validation, y_pred)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)
pfr = fp / (fp + tn)
fnr = fn / (fn + tp)

brier_score = brier_score_loss(y_validation, y_prob)
print(f"Brier Score: {brier_score:.4f}")
aurpc = average_precision_score(y_validation, y_prob)
print(f"AURPC: {aurpc:.4f}")

# Print all metrics
print(f"ROC AUC: {roc_auc:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Positive Predictive Value (PPV): {ppv:.4f}")
print(f"Negative Predictive Value (NPV): {npv:.4f}")
print(f"Positive False Rate (PFR): {pfr:.4f}")
print(f"False Negative Rate (FNR): {fnr:.4f}")

In [ ]:
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:,1]
cm=confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
roc_auc=roc_auc_score(y_test, y_prob)
f1 = f1_score(y_test,y_pred)
accuracy = accuracy_score(y_test, y_pred)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)
pfr = fp / (fp + tn)
fnr = fn / (fn + tp)

brier_score = brier_score_loss(y_test, y_prob)
print(f"Brier Score: {brier_score:.4f}")
aurpc = average_precision_score(y_test, y_prob)
print(f"AURPC: {aurpc:.4f}")

# Print all metrics
print(f"ROC AUC: {roc_auc:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Positive Predictive Value (PPV): {ppv:.4f}")
print(f"Negative Predictive Value (NPV): {npv:.4f}")
print(f"Positive False Rate (PFR): {pfr:.4f}")
print(f"False Negative Rate (FNR): {fnr:.4f}")

In [ ]:
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_validation)
shap_importance = np.abs(shap_values[1]).mean(axis=0)  # Calculate mean absolute SHAP value for each feature
importance_order = np.argsort(shap_importance)[::-1]  # Sort features by importance in descending order
top_15_idx = importance_order[:15]
top_15_shap_values = shap_values[1][:, top_15_idx]
top_15_features = X_validation.columns[top_15_idx]
shap.summary_plot(top_15_shap_values, X_validation.iloc[:, top_15_idx], feature_names=top_15_features)